In [60]:
import httpx
from decouple import config
from decouple import Config, RepositoryEnv, config
import pathlib

NBS_DIR = pathlib.Path().resolve()
REPO_DIR = NBS_DIR.parent

config_env = Config(RepositoryEnv(REPO_DIR / '.env'))

API_ACCESS_KEY = config_env('API_ACCESS_KEY')

headers = {
    "X-API-Key" : API_ACCESS_KEY
}
endpoint = "http://127.0.0.1:8000/"
# prompt = "a phot of TOK who is a architecture explaining building plans to a contractor"
res = httpx.get(endpoint, headers=headers)
res.status_code, res.json(), res.headers.get('Retry-After')

(200, {'Hello': 'World'}, None)

In [32]:
r

NameError: name 'r' is not defined

In [30]:
res.headers.get("X-Forwarded-For")